In [ ]:
#%matplotlib inline
import d2ltvm
import inspect
from IPython import display
import numpy as np
import timeit
import tvm
from tvm import te
from matplotlib import pyplot as plt

In [ ]:
def plot(X, Y, xlabel=None, ylabel=None, legend=[], xlim=None,
         ylim=None, xscale='linear', yscale='linear', fmts=None,
         figsize=(4.5, 3)):
    """Plot multiple lines"""
    display.set_matplotlib_formats('svg')
    plt.rcParams['figure.figsize'] = figsize
    axes = plt.gca()
    X, Y = np.array(X), np.array(Y)
    if X.shape != Y.shape: X = [X] * len(Y)
    if not fmts: fmts = ['-'] * len(X)
    for x, y, fmt in zip(X, Y, fmts):
        axes.plot(x, y, fmt)
    axes.set_xlabel(xlabel)
    axes.set_ylabel(ylabel)
    axes.set_xscale(xscale)
    axes.set_yscale(yscale)
    axes.set_xlim(xlim)
    axes.set_ylim(ylim)
    if legend: axes.legend(legend)
    axes.grid()

def plot_gflops(sizes, gflops, legend, xlabel='Size'):
    plot(sizes, gflops, xlabel=xlabel, ylabel='GFLOPS',
             xscale='log', yscale='log',
             legend=legend, fmts=['--']*(len(gflops)-1)+['-'])
    
def bench_workload(workload):
    workload(1)
    time = workload(1)
    if time > 1: return time
    num_repeats = max(int(1.0/time),5)
    return workload(num_repeats)/num_repeats

In [ ]:

sizes = 2**np.arange(5,29,4)
np_add = lambda n: timeit.Timer(setup='import numpy as np\n'
            'import d2ltvm\n'
            'a,b,c=d2ltvm.get_abc(%d)'%n,
            stmt='np.add(a,b,out=c)')

exe_times=[bench_workload(np_add(n).timeit) for n in sizes]
np_gflops=sizes / 1e9 / np.array(exe_times)
plot_gflops(sizes, [np_gflops],['numpy'])

### Default Schedule

In [ ]:
def default(n):
    A,B,C=d2ltvm.vector_add(n)
    s=te.create_schedule(C.op)
    return s,(A,B,C)
s,args = default(64)
print(tvm.lower(s,args,simple_mode=True))

In [ ]:
target = 'llvm -mcpu=skylake-avx512'
mod = tvm.build(s, arg, target)
print(mod.get_source()[:500])